In [2]:
from google.colab import userdata

In [3]:
!pip install -q langchain-community langchain_google_genai langchain_huggingface \ faiss-cpu tiktoken python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.1/719.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.
google-colab

In [4]:
!pip install -U sentence-transformers

In [5]:
!pip install -U langchain_FAISS

In [6]:
!pip install youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 9.4 MB/s eta 0:00:00


In [7]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

---
# Indexing

In [9]:
video_id = "LPZh9BOjkQs"

try:
   transcript_list = YouTubeTranscriptApi().fetch(video_id=video_id, languages=["en"])

   transcript = " ".join(chunk.text for chunk in transcript_list)
   print(transcript)

except TranscriptsDisabled:
  print("Some error occured")

Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical AI assistant, add on whatever the use

In [10]:
len(transcript)

7524

---
# Text Splitting

In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks=splitter.create_documents([transcript])
print(len(chunks))

10


---
# Vector Embeddings

In [12]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
vector_store = FAISS.from_documents(chunks, embedding)

In [14]:
vector_store.index_to_docstore_id

{0: '80163a82-0bbf-4a47-9e8c-752cd56d9f02',
 1: 'ac29f2be-ef62-423e-85a1-226456108226',
 2: '3b2be00b-346a-4d36-b991-094b27b0ec43',
 3: '127e6caa-ab00-45f1-aea6-b9507c58cf78',
 4: '2098f49a-edee-41de-80c0-d2c5930f8735',
 5: '60834f89-94ad-4f83-8114-f445391e4e79',
 6: '6c9a1d6e-495c-4d98-aa0a-19fead529d69',
 7: 'c0c87150-8be8-441d-8a6b-06929bc0201d',
 8: 'bc594cbc-1229-42db-bebe-2675283b88ad',
 9: 'e665c394-2021-4474-9a80-67e2d6e4a3ed'}

In [16]:
vector_store.get_by_ids(['e665c394-2021-4474-9a80-67e2d6e4a3ed'])

[Document(id='e665c394-2021-4474-9a80-67e2d6e4a3ed', metadata={}, page_content='the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.')]

---
# Retriever

In [17]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={"k":4})

In [18]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b73f0b5aa20>, search_kwargs={'k': 4})

In [19]:
retriever.invoke("What is a large language model")

[Document(id='3b2be00b-346a-4d36-b991-094b27b0ec43', metadata={}, page_content="a standard human to read the amount of text that was used to train GPT-3, for example, if they read non-stop 24-7, it would take over 2600 years. Larger models since then train on much, much more. You can think of training a little bit like tuning the dials on a big machine. The way that a language model behaves is entirely determined by these many different continuous values, usually called parameters or weights. Changing those parameters will change the probabilities that the model gives for the next word on a given input. What puts the large in large language model is how they can have hundreds of billions of these parameters. No human ever deliberately sets those parameters. Instead, they begin at random, meaning the model just outputs gibberish, but they're repeatedly refined based on many example pieces of text. One of these training examples could be just a handful of words, or it could be thousands,

---
# Augmentation(joining the query and context in prompt)

In [20]:
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [21]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0.2)

In [22]:
prompt = PromptTemplate(
    template="""
    You are a helpful AI assitant,
    Answer ONLY from the provided transcript context.
    If the context is insufficient, just say you don't know.

    Context: {context}
    Question: {question}
    """,
    input_variables=['context', 'question']
)

In [23]:
question = "What is an llm?"
retrieved_docs = retriever.invoke(question)

In [24]:
retrieved_docs

[Document(id='c0c87150-8be8-441d-8a6b-06929bc0201d', metadata={}, page_content="type of operation known as a feed-forward neural network, and this gives the model extra capacity to store more patterns about language learned during training. All of this data repeatedly flows through many different iterations of these two fundamental operations, and as it does so, the hope is that each list of numbers is enriched to encode whatever information might be needed to make an accurate prediction of what word follows in the passage. At the end, one final function is performed on the last vector in this sequence, which now has had a chance to be influenced by all the other context from the input text, as well as everything the model learned during training, to produce a prediction of the next word. Again, the model's prediction looks like a probability for every possible next word. Although researchers design the framework for how each of these steps work, it's important to understand that the s

In [25]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
print(context_text)

type of operation known as a feed-forward neural network, and this gives the model extra capacity to store more patterns about language learned during training. All of this data repeatedly flows through many different iterations of these two fundamental operations, and as it does so, the hope is that each list of numbers is enriched to encode whatever information might be needed to make an accurate prediction of what word follows in the passage. At the end, one final function is performed on the last vector in this sequence, which now has had a chance to be influenced by all the other context from the input text, as well as everything the model learned during training, to produce a prediction of the next word. Again, the model's prediction looks like a probability for every possible next word. Although researchers design the framework for how each of these steps work, it's important to understand that the specific behavior is an emergent phenomenon based on how those hundreds of

it do

In [26]:
final_prompt = prompt.invoke({"context":context_text, "question":question})

print(final_prompt)

text="\n    You are a helpful AI assitant,\n    Answer ONLY from the provided transcript context.\n    If the context is insufficient, just say you don't know.\n\n    Context: type of operation known as a feed-forward neural network, and this gives the model extra capacity to store more patterns about language learned during training. All of this data repeatedly flows through many different iterations of these two fundamental operations, and as it does so, the hope is that each list of numbers is enriched to encode whatever information might be needed to make an accurate prediction of what word follows in the passage. At the end, one final function is performed on the last vector in this sequence, which now has had a chance to be influenced by all the other context from the input text, as well as everything the model learned during training, to produce a prediction of the next word. Again, the model's prediction looks like a probability for every possible next word. Although researcher

---
# Generating answers

In [27]:
answer = llm.invoke(final_prompt)

In [28]:
print(answer.content)

A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text.


---
# Chains

In [29]:
from langchain_core.runnables import RunnableLambda,RunnableParallel,RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [30]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [32]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [34]:
parser = StrOutputParser()

In [35]:
main_chain = parallel_chain | prompt | llm | parser

In [39]:
output = main_chain.invoke("Summarise this video for me")
print(output)

I don't know.


---
---
# Final Working

In [42]:
while True:
  user_input = input("\nEnter your question: ")
  if(user_input == 'quit'):
    print("Exiting the code")
    break
  else:
    rag_ouput = main_chain.invoke(user_input)
    print(f"Answer: {rag_ouput}\n")


Enter your question: what is an llm
Answer: A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text.


Enter your question: how does llm work
Answer: A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical AI assistant, add on whatever the user types in as the first part of the interaction, and then have the model repeatedly predict the next word that such a hypothetical AI assistant would say in response. This process involves data repeatedly flowing through many different iterations of fundamental operations like feed-forward neural networks, enriching lists of numbers to encode information needed for accurate predictions. A final f